In [1]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import arviz as az

from tbdynamics.calibration.plotting import plot_spaghetti, plot_output_ranges
from tbdynamics.tools.inputs import load_targets
from tbdynamics.settings import CM_PATH, OUT_PATH, DOCS_PATH
from tbdynamics.constants import compartments
import pickle
import pandas as pd
import plotly.express as px
import numpy as np
from  tbdynamics.camau.calibration.utils import get_bcm
from tbdynamics.constants import quantiles
from tbdynamics.camau.constants import indicator_legends, indicator_names
from estival.sampling import tools as esamp


WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# loaded_inference_data = az.from_netcdf(OUT_PATH / 'inference_data1.nc')
# idata = az.from_netcdf(OUT_PATH / 'extracted_idata.nc')
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
targets = load_targets(CM_PATH / "targets.yml")

In [3]:
idata_raw = az.from_netcdf(OUT_PATH / 'camau/r0303/calib_full_out.nc')

In [4]:
burnt_idata = idata_raw.sel(draw=np.s_[1000:])

In [5]:
idata_extract = az.extract(burnt_idata, num_samples=200)

In [6]:
idata_extract

<xarray.Dataset>
Dimensions:                       (sample: 200)
Coordinates:
  * sample                        (sample) object MultiIndex
  * chain                         (sample) int32 0 3 3 2 2 1 2 ... 1 0 1 2 2 2 3
  * draw                          (sample) int32 1670 1363 1164 ... 1838 1930
Data variables: (12/15)
    contact_rate                  (sample) float64 0.04607 0.01315 ... 0.0116
    rr_infection_latent           (sample) float64 0.08634 0.2693 ... 0.2092
    rr_infection_recovered        (sample) float64 0.2043 0.07301 ... 0.05868
    progression_multiplier        (sample) float64 0.9278 1.558 ... 1.655 1.906
    smear_positive_death_rate     (sample) float64 0.3915 0.3553 ... 0.3509
    smear_negative_death_rate     (sample) float64 0.01851 0.0195 ... 0.01865
    ...                            ...
    screening_inflection_time     (sample) float64 1.988e+03 ... 1.998e+03
    time_to_screening_end_asymp   (sample) float64 3.431 2.085 ... 3.144 2.075
    acf_sensitivity               (sample) float64 0.8883 0.8559 ... 0.9766
    detection_reduction           (sample) float64 0.2273 0.5911 ... 0.5334
    notif_dispersion              (sample) float64 106.3 143.0 ... 104.0 148.8
    latent_dispersion             (sample) float64 9.548 6.614 ... 9.444 7.397
Attributes:
    created_at:                 2025-03-03T05:44:01.766369
    arviz_version:              0.17.0
    inference_library:          pymc
    inference_library_version:  5.2.0
    sampling_time:              763.6157789230347
    tuning_steps:               1000

In [7]:
# outputs = calculate_scenario_outputs(params, idata)
# with open(OUT_PATH / 'quant_outputs.pkl', 'wb') as f:
#      pickle.dump(outputs, f)

In [ ]:
# with open(OUT_PATH /'camau/quant_outputs.pkl', 'rb') as f:
#     outputs = pickle.load(f)
scenario_config = {"detection_reduction": True, "contact_reduction": False}

    # Base scenario (calculate outputs for all indicators)
bcm = get_bcm(params, scenario_config, homo_mixing=False)
base_results = esamp.model_results_for_samples(idata_extract, bcm).results
base_quantiles = esamp.quantiles_for_results(base_results, quantiles)

In [ ]:
base_quantiles

In [ ]:
plot_output_ranges(base_quantiles,targets,['percentage_latent'],indicator_names,indicator_legends,1,2010,2025)

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets1.png", scale=3)
plot_output_ranges(base_quantiles,targets,['mortality'],indicator_names,indicator_legends,1,2010,2025)

In [ ]:
plot_output_ranges(base_quantiles,targets,['total_population'],indicator_names,indicator_legends,1,2010,2025)


In [ ]:
plot_output_ranges(base_quantiles,targets,['notification'],indicator_names,indicator_legends,1,2010,2025)

In [ ]:
plot_output_ranges(base_quantiles,targets,['incidence'],indicator_names,indicator_legends,1,2010,2025)


In [ ]:
plot_output_ranges(base_quantiles,targets,['acf_detectionXact3_trialXorgan_pulmonary'],indicator_names,indicator_legends,1,2014,2019)

In [ ]:
plot_output_ranges(base_quantiles,targets,['acf_detectionXact3_controlXorgan_pulmonary'],indicator_names,indicator_legends,1,2010,2025)

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets2.png", scale=3)

In [ ]:
# spah.write_image(DOCS_PATH / 'spah.png', scale = 3)

In [ ]:
# target_plot_history = plot_output_ranges(outputs['base_scenario'],targets,['total_population','notification','adults_prevalence_pulmonary'],1,1800,2010, history =True)

In [ ]:
# target_plot_history

In [ ]:
# target_plot_history.write_image(DOCS_PATH / 'targets_history.png', scale=3)

In [ ]:
# compare_target_plot = plot_output_ranges(outputs['base_scenario'],targets,['incidence','mortality_raw','prevalence_smear_positive', 'percentage_latent'],2,2010,2025)

In [ ]:
# compare_target_plot.write_image(DOCS_PATH / "non_targets.png", scale='3')

In [ ]:
# compare_target_plot

In [ ]:
# screening_plot.write_image(DOCS_PATH / 'screening_plot.png', scale =3)

In [ ]:
cdr_plot = plot_output_ranges(outputs['base_scenario']['quantiles'],targets,['case_notification_rate'],1,2010,2025)

In [ ]:
cdr_plot

In [ ]:
# cdr_plot.write_image(DOCS_PATH / 'cdr_plot.png', scale =3)

In [ ]:
early_plot = plot_output_ranges(outputs['base_scenario']['quantiles'],targets,['incidence_early_prop'] ,1,2010,2025)

In [ ]:
early_plot

In [ ]:
# early_plot.write_image(DOCS_PATH / 'early_plot.png', scale =3)

In [ ]:
# # Simulate data extraction, replace with actual data extraction
# data_early = outputs['base_scenario']['quantiles']['incidence_early_prop'][0.5].reset_index()
# data_early['type'] = 'Early incidence'

# data_late = outputs['base_scenario']['quantiles']['incidence_late_prop'][0.5].reset_index()
# data_late['type'] = 'Late incidence'

# # Combine dataframes
# data_combined = pd.concat([data_early, data_late])
# fig = px.area(data_combined, x='time', y=0.5, color='type',
#               labels={0.5: 'Proportion (%)', 'time': ''},
#               title='',
#               range_x=[2000, 2025], range_y=[0,100])  # Update x-axis range

# # Update layout to customize legend and axis range
# fig.update_layout(
#     xaxis=dict(
#         title='',  # No x-axis title specified
#         title_font=dict(size=12)  # Setting size even if title is not shown
#     ),
#     yaxis=dict(
#         title='Incidence Proportion',
#         title_font=dict(size=12)  # Specific font size for y-axis title
#     ),
#     legend_title_text='',  # Remove legend title
#     legend=dict(
#         orientation="h",  # Horizontal legend
#         yanchor="bottom",
#         y=-0.15,  # Position the legend below the plot
#         xanchor="center",
#         x=0.5,
#         font=dict(size=12)  # Font size for legend items
#     ),
#     margin=dict(l=20, r=20, t=20, b=50),  # Adjusted margins for better spacing
#     font=dict(  # Apply font size adjustments
#         family="Arial, sans-serif",  # Set font family
#         size=12,  # Default font size for other text elements
#         color="black"  # Font color
#     )
# )

# fig.show()

In [ ]:
# base_df = outputs['base_scenario']['quantiles']

# # Calculate 80% of the original values for `prop_early_latent` and `prop_late_latent`


# # Extract data for each compartment, label it, and store in a list
# data_frames = []
# for compartment in compartments:
#     if f'prop_{compartment}' in base_df.columns:
#         df = base_df[f'prop_{compartment}'][0.5].reset_index()  # Adjust this if your quantile structure is different
#         df[0.5] *= 100  # Convert to percentage
#         df['type'] = compartment.replace('_', ' ').capitalize()
#         data_frames.append(df)

# # Combine all compartment data into one DataFrame for plotting
# combined_data = pd.concat(data_frames)

# # Plot using Plotly Express
# fig = px.area(combined_data, x='time', y=0.5, color='type',
#               labels={'0.5': 'Proportion (%)', 'time': 'Time'},
#               title='', range_x=[1980, 2025], range_y=[0, 100])

# # Update layout
# fig.update_layout(
#     xaxis=dict(title='', title_font=dict(size=12)),
#     yaxis=dict(title='<b>Proportion (%)</b>', title_font=dict(size=12), title_standoff=0),
#     legend_title_text='',
#     legend=dict(
#         orientation="h",
#         yanchor="bottom",
#         y=-0.2,
#         xanchor="center",
#         x=0.5,
#         font=dict(size=12)
#     ),
#     height=320,  # Set the figure height
#     margin=dict(l=10, r=5, t=10, b=40),
#     font=dict(family="Arial, sans-serif", size=12, color="black")
# )

# # Show the plot
# fig.show()


In [ ]:
# fig.write_image(DOCS_PATH / 'comps.png', scale =3)